# Task 1: Feature Engineering with Modin on Ray
In this task, we'll perform some feature engineering for the Amazon Reviews dataset. We'll use Modin so that you can conveniently write pandas code and have your workload scale to a cluster.

In [1]:
import ray
ray.init()
import os 
import json
import modin.pandas as pd
import numpy as np
from ast import literal_eval

2024-03-11 20:11:37,932	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-03-11 20:11:37,942	INFO worker.py:1540 -- Connecting to existing Ray cluster at address: 10.41.0.13:6380...
2024-03-11 20:11:37,979	INFO worker.py:1715 -- Connected to Ray cluster. View the dashboard at http://10.41.0.13:8265 


In [26]:
ray.shutdown()

In [2]:
ROOT = "~/public/pa3/"
ROOT = os.path.expanduser(ROOT) # expand user contractions

Optional cleaning up: This is helpful when you validate your notebook in the end

In [3]:
!rm -f res_1_1.json res_1_2.json res_1_3.json res_1_4.json # remove previously saved results

In [4]:
def read_csv(directory):
    files = [file for file in os.listdir(directory) if file.endswith('.csv')]
    files = [os.path.join(directory, file) for file in files]
    return pd.concat(map(pd.read_csv, files))

In [5]:
product_data = read_csv(os.path.join(ROOT, "metadata_header.csv"))
# add optional preprocessing here to make sure your columns are of the right dtypes
# YOUR CODE HERE
product_data['categories'] = product_data['categories'].apply(lambda x: literal_eval(x) if pd.notna(x) else None)
product_data['salesRank'] = product_data['salesRank'].apply(lambda x: literal_eval(x) if pd.notna(x) else None)
product_data['related'] = product_data['related'].apply(lambda x: literal_eval(x) if pd.notna(x) else None)

In [6]:
product_data.head(10)

,asin,salesRank,categories,title,price,related
0,1590927052,{'Books': 7246388},[[Books]],"The Blackhope Scaur Trilogy, Book Two: The Aba...",NaN,None
1,1590923332,{'Books': 12402333},[[Books]],The Short Division of the World Secret Service...,NaN,None
2,1590921437,{'Books': 3398366},[[Books]],Weddings and Courtships - France,NaN,None
3,1590950119,{'Books': 12650149},[[Books]],InsideScoop to MCP/MCSE Exam 70-291 Windows Se...,NaN,None
4,1590941594,{'Books': 8880497},[[Books]],Murder at the Blue Ridge Barbecue Festival,11.24,None
5,1590889975,{'Books': 12121372},[[Books]],The House in the Steeple,3.99,None
6,1590950305,{'Books': 9207098},[[Books]],ExamInsight For MCP/MCSE Exam 70-291 Windows S...,32.13,None
7,1590925939,{'Books': 11788357},[[Books]],"The Dark Sorcerer's Return, Book Three of the ...",NaN,None
8,1590925920,{'Books': 11654621},[[Books]],David and Dad Catch the Rainbow,NaN,None
9,1590930053,{'Books': 1851853},[[Books]],Outside (Molly Book),8.96,"{'also_bought': ['1590930045', '1590930061', '..."


# Task 1.1: flatten `categories` and `salesRank`

In [7]:
def part1(product_data):
    
    # Column names
    salesRank_column = 'salesRank'
    categories_column = 'categories'
    asin_column = 'asin'
    
    # Outputs
    category_column = 'category'
    bestSalesCategory_column = 'bestSalesCategory'
    bestSalesRank_column = 'bestSalesRank'
    
    res = {
        'count_total': None,
        'mean_bestSalesRank': None,
        'variance_bestSalesRank': None,
        'numNulls_category': None,
        'countDistinct_category': None,
        'numNulls_bestSalesCategory': None,
        'countDistinct_bestSalesCategory': None
    }
    # populate res below
    # YOUR CODE HERE
    product_data[category_column] = product_data[categories_column].apply(lambda x: x[0][0] if x and x[0] and len(x[0][0])>0 else None)

    # Task 2: Flatten the salesRank column into two separate columns
    product_data[bestSalesCategory_column] = product_data[salesRank_column].apply(lambda x: list(x.keys())[0] if x else None)
    product_data[bestSalesRank_column] = product_data[salesRank_column].apply(lambda x: list(x.values())[0] if x else None)
        
    res['count_total'] = int(product_data.shape[0])
    res['mean_bestSalesRank'] = float(product_data[bestSalesRank_column].mean())
    res['variance_bestSalesRank'] = float(product_data[bestSalesRank_column].var())
    res['numNulls_category'] = int(product_data[category_column].isna().sum())
    res['countDistinct_category'] = int(product_data[category_column].nunique())
    res['numNulls_bestSalesCategory'] = int(product_data[bestSalesCategory_column].isna().sum())
    res['countDistinct_bestSalesCategory'] = int(product_data[bestSalesCategory_column].nunique())

    return res


In [8]:
res = part1(product_data)
print(res)

{'count_total': 1960789, 'mean_bestSalesRank': 1751359.4469592853, 'variance_bestSalesRank': 5632665706967.55, 'numNulls_category': 32180, 'countDistinct_category': 75, 'numNulls_bestSalesCategory': 306918, 'countDistinct_bestSalesCategory': 33}


In [9]:
with open("res_1_1.json", "w") as f:
    json.dump(res, f)

# Task 1.2: flatten `related`

In [11]:
def part2(product_data):
    #Column names
    asin_column = 'asin'
    price_column = 'price'
    attribute = 'also_viewed'
    related_column = 'related'
    
    #Outputs
    meanPriceAlsoViewed_column = 'meanPriceAlsoViewed'
    countAlsoViewed_column = 'countAlsoViewed'
    
    res = {
        'count_total': None,
        'mean_meanPriceAlsoViewed': None,
        'variance_meanPriceAlsoViewed':  None,
        'numNulls_meanPriceAlsoViewed':  None,
        'mean_countAlsoViewed':  None,
        'variance_countAlsoViewed':  None,
        'numNulls_countAlsoViewed':  None
    }
    
    # populate res below
    # YOUR CODE HERE
    asin_price_dict = dict(zip(product_data['asin'], product_data['price']))
    
    # Function to calculate mean price for a given list of product IDs
    def calculate_mean_price(product_ids, asin_price_dict):
        valid_prices = [asin_price_dict.get(pid) for pid in product_ids if pid in asin_price_dict]
        valid_prices = [price for price in valid_prices if pd.notna(price)]
        return sum(valid_prices) / len(valid_prices) if len(valid_prices) > 0 else None

    # Apply the function to the DataFrame, handling the case when 'also_viewed' is not present
#     product_data['meanPriceAlsoViewed'] = product_data['related'].apply(lambda x: calculate_mean_price(x.get('also_viewed', []), asin_price_dict) if isinstance(x, dict) else None)
    product_data['meanPriceAlsoViewed'] = product_data['related'].apply(lambda x: calculate_mean_price(x.get('also_viewed', []), asin_price_dict) if isinstance(x, dict) else None)

    # Apply similar logic to the 'countAlsoViewed'
    product_data[countAlsoViewed_column] = product_data[related_column].apply(lambda row: len(row.get(attribute, [])) if isinstance(row, dict) and row.get(attribute) else None)
    
    # Calculate required statistics and update the res dictionary
    res['count_total'] = int(product_data.shape[0])
    res['mean_meanPriceAlsoViewed'] = float(product_data[meanPriceAlsoViewed_column].mean()) if product_data[meanPriceAlsoViewed_column].mean() is not np.nan else None
    res['variance_meanPriceAlsoViewed'] = float(product_data[meanPriceAlsoViewed_column].var()) if product_data[meanPriceAlsoViewed_column].var() is not np.nan else None
    res['numNulls_meanPriceAlsoViewed'] = int(product_data[meanPriceAlsoViewed_column].isnull().sum())
    res['mean_countAlsoViewed'] = float(product_data[countAlsoViewed_column].mean()) if product_data[countAlsoViewed_column].mean() is not np.nan else None
    res['variance_countAlsoViewed'] = float(product_data[countAlsoViewed_column].var()) if product_data[countAlsoViewed_column].var() is not np.nan else None
    res['numNulls_countAlsoViewed'] = int(product_data[countAlsoViewed_column].isnull().sum())

    
    return res

In [12]:
res = part2(product_data)
print(res)

{'count_total': 1960789, 'mean_meanPriceAlsoViewed': 37.63668174891553, 'variance_meanPriceAlsoViewed': 4392.26524199995, 'numNulls_meanPriceAlsoViewed': 1531568, 'mean_countAlsoViewed': 28.337187114866374, 'variance_countAlsoViewed': 571.6873965669193, 'numNulls_countAlsoViewed': 1330326}


In [14]:
with open("res_1_2.json", "w") as f:
    json.dump(res, f)

# Task 1.3: impute `price`

In [15]:
from sklearn.impute import SimpleImputer



def part3(product_data):

    price_column = 'price'
    title_column = 'title'
    # Outputs:
    meanImputedPrice_column = 'meanImputedPrice'
    medianImputedPrice_column = 'medianImputedPrice'
    unknownImputedTitle_column = 'unknownImputedTitle'
    
    res = {
        'count_total': None,
        'mean_meanImputedPrice': None,
        'variance_meanImputedPrice': None,
        'numNulls_meanImputedPrice': None,
        'mean_medianImputedPrice': None,
        'variance_medianImputedPrice': None,
        'numNulls_medianImputedPrice': None,
        'numUnknowns_unknownImputedTitle': None,
    }
    # Populate res below
    # YOUR CODE HERE
    mean_imputer = SimpleImputer(strategy='mean')
    product_data[meanImputedPrice_column] = mean_imputer.fit_transform(product_data[[price_column]])

    # Impute missing values in the 'price' column with median
    median_imputer = SimpleImputer(strategy='median')
    product_data[medianImputedPrice_column] = median_imputer.fit_transform(product_data[[price_column]])

    # Impute missing values in the 'title' column with 'unknown'
    product_data[unknownImputedTitle_column] = product_data[title_column].fillna('unknown')

    # Calculate statistics
    res['count_total'] = int(len(product_data))
    res['mean_meanImputedPrice'] = float(product_data[meanImputedPrice_column].mean())
    res['variance_meanImputedPrice'] = float(product_data[meanImputedPrice_column].var())
    res['numNulls_meanImputedPrice'] = int(product_data[meanImputedPrice_column].isnull().sum())
    res['mean_medianImputedPrice'] = float(product_data[medianImputedPrice_column].mean())
    res['variance_medianImputedPrice'] = float(product_data[medianImputedPrice_column].var())
    res['numNulls_medianImputedPrice'] = int(product_data[medianImputedPrice_column].isnull().sum())
    res['numUnknowns_unknownImputedTitle'] = int((product_data[unknownImputedTitle_column] == 'unknown').sum())

    return res

In [16]:
res = part3(product_data)
print(res)

{'count_total': 1960789, 'mean_meanImputedPrice': 26.58890293679852, 'variance_meanImputedPrice': 2075.742678711281, 'numNulls_meanImputedPrice': 0, 'mean_medianImputedPrice': 22.923684200594764, 'variance_medianImputedPrice': 2110.7956911074302, 'numNulls_medianImputedPrice': 0, 'numUnknowns_unknownImputedTitle': 137073}


In [17]:
with open("res_1_3.json", "w") as f:
    json.dump(res, f)

# Task 1.4: Process `title` and one-hot encode `category`

For this task, we'll use the preprocessed dataset `product_processed.csv`. Our goal is to split up the `title` entries into constituent words and one-hot encode the `category` column.

In [18]:
del product_data # free up some memory

In [19]:
product_processed_data = pd.read_csv(os.path.join(ROOT, "product_processed.csv")) # read a single csv file

1. For each row, convert `title` to lowercase, then split it by whitespace (’ ’) to an array of strings, store this array in a new column titleArray.
2. One-hot encode the `category` column. In this case, make sure to use the sorted order of the unique categories to form the one-hot vector.

In [20]:
def part4(product_processed_data):
    
    # Inputs
    title_column = 'title'
    category_column = 'category'
    
    # Outputs
    titleArray_column = 'titleArray'
    categoryOneHot_column = 'categoryOneHot'
    
    res = {
        'count_total': None, 
        'meanLength_titleArray': None,
        'mean_categoryOneHot': None,
    }
    # populate res below
     # Step 1: Convert 'title' to lowercase and split into an array of strings
    product_processed_data[titleArray_column] = product_processed_data[title_column].str.lower().str.split(' ')

    # Step 2: Calculate mean length of the arrays in 'titleArray'
    res['meanLength_titleArray'] = product_processed_data[titleArray_column].apply(len).mean()

    # Step 3: One-hot encode the 'category' column
    categories = sorted(product_processed_data[category_column].unique())
    one_hot_encoded = pd.get_dummies(product_processed_data[category_column], columns=categories)
    product_processed_data = pd.concat([product_processed_data, one_hot_encoded], axis=1)

    # Calculate the mean value of one-hot encoding vectors
    mean_categoryOneHot = np.mean(one_hot_encoded, axis=0).tolist()
    res['mean_categoryOneHot'] = mean_categoryOneHot

    # Calculate total count
    res['count_total'] = len(product_processed_data)

    return res

In [21]:
res = part4(product_processed_data)
print(res)

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


{'count_total': 9430000, 'meanLength_titleArray': 8.099168716861081, 'mean_categoryOneHot': [1.0604453870625663e-07, 0.0006977730646871687, 2.226935312831389e-06, 0.0008027571580063626, 0.00040943796394485684, 4.241781548250265e-07, 0.002560445387062566, 3.181336161187699e-07, 0.003249946977730647, 0.0012358430540827148, 0.006526405090137858, 1.6118769883351007e-05, 0.012449946977730646, 0.03510593849416755, 0.0075626723223753975, 0.0010625662778366914, 0.02748356309650053, 9.544008483563096e-05, 0.25131601272534465, 3.022269353128314e-05, 0.0002009544008483563, 0.05214347826086956, 4.899257688229056e-05, 4.32661717921527e-05, 2.1208907741251325e-07, 0.0367677624602333, 3.297985153764581e-05, 0.0007919406150583245, 3.340402969247084e-05, 0.00024422057264050903, 0.1522659597030753, 1.0604453870625663e-07, 0.0006481442205726405, 0.000471898197242842, 0.00015238600212089077, 0.0004906680805938494, 6.256627783669141e-06, 0.028671261930010603, 0.05254252386002121, 6.362672322375398e-07, 0.0

In [22]:
with open("res_1_4.json", "w") as f:
    json.dump(res, f)

# Is that it?

To make this dataset ready for ML tasks, there are many more feature engineering steps to be done, which we haven't covered here. For example, the `titleArray` column can now be fed into a Word2Vec model for meaningful embeddings. We've stuck with a few simple operations here. Next, in task 2, you'll use preprocessed train and test datasets to train Xgboost models in a multi-node fashion!

In [ ]:
# shutdown!
ray.shutdown()